<p style="background-color:purple;font-family:newtimeroman;font-size:200%;color:white;text-align:center;border-radius:60px 20px;"><b>Word Embedding - Bert Multiclass Data</b></p>

**If your computer is not GPU compatible, I recommend you run this notebook on Kaggle.**

# <font color='purple'> <b>Importing Libraries</b><font color='black'>  

In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/haber-sitesi/Projem.xlsx
/kaggle/input/deneme/denemem.xlsx


SimpleTransformers is a Python library that allows you to easily use Transformer-based models (such as BERT, GPT, RoBERTa, etc.).

The library is built on top of Hugging Face Transformers and provides a simple API, especially for natural language processing (NLP) projects. This allows you to train, fine-tune, and use complex models with minimal code.

[Simple Transformers Link](https://simpletransformers.ai/)

In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 829.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=49ea999c20841f0f1fce1f6b7a54a85b6e47b73052517f7c5585373019fec3b6
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
from simpletransformers.classification import ClassificationModel

# <font color='purple'> <b>Loading The Dataset</b><font color='black'>  

In [4]:
a=pd.read_excel("/kaggle/input/bert-multiclass/bert_dataset.xlsx")

* Our dataset contains approximately 3,000 news articles from news websites, categorized into five categories.

* Categories: Science and Technology, Economy, Health, Politics, Sports

In [5]:
a

,kategori,metin
0,Bilim Ve Teknoloji,Fransa Ulusal Arkeolojik Araştırma Enstitüsü’n...
1,Bilim Ve Teknoloji,"Baykar Teknoloji Lideri Selçuk Bayraktar, Kana..."
2,Bilim Ve Teknoloji,"Soyuz MS-18 uzay aracı, Kazakistan’ın Baykonur..."
3,Bilim Ve Teknoloji,"Siyasetten ekonomiye, spordan sağlığa, özel ha..."
4,Bilim Ve Teknoloji,Öğrencilerin uzay teknolojileri ve roket bilim...
...,...,...
3056,Spor,UEFA Avrupa Ligi'nde 2016-2017 sezonu grup kur...
3057,Spor,Spor Toto Süper Lig'in 2. haftasında görev ala...
3058,Spor,Geçen sezonun sona ermesinin ardından Trabzons...
3059,Spor,"TFF'den yapılan açıklamada, daha önce kurulda ..."


In [6]:
a["kategori"].unique()

array(['Bilim Ve Teknoloji', 'Ekonomi', 'Sağlık', 'Siyaset', 'Spor'],
      dtype=object)

**English Names of The Categories**
- Bilim Ve Teknoloji - Science and Technology
- Ekonomi - Economy
- Sağlık - Health
- Siyaset - Politics
- Spor - Sprts


In [7]:
a['labels'] = pd.factorize(a.kategori)[0]

# <font color='purple'> <b>Train | Test Split</b><font color='black'>  

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(a, test_size=0.2, random_state=42)

In [9]:
train=train[["metin","labels"]]
test=test[["metin","labels"]]

In [10]:
#for bert text = string      label = int
train["metin"]=train["metin"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

**SimpleTransformers** automates many data preprocessing and model training tasks by utilizing the Hugging Face Transformers library in the background. This means you don’t have to manually perform many steps, and you can quickly prepare your dataset for the BERT model.

**SimpleTransformers** handles the basic text preprocessing steps required for BERT and similar transformer-based models. The text data needs to go through the following processes:
* Tokenization: The text is split into chunks (tokens) to make it compatible with the BERT model. This process is done using the BERT Tokenizer.

* Padding and Truncation: Since input texts need to be of a fixed length, shorter texts are extended with padding, and longer texts are truncated to a specific length.

* BERT Input IDs and Attention Masking: Each token is converted into an integer ID. Additionally, an attention_mask is created to indicate which tokens the model should focus on.

These processes are automatically handled within SimpleTransformers. All you need to do is provide the text as a string, and the library takes care of the rest.

# <font color='purple'> <b>Modeling</b><font color='black'>  

In [11]:
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=5, use_cuda=True,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 
                                  'num_train_epochs': 5, "train_batch_size": 64 , "fp16":False, 
                                  "output_dir": "bert_model", "save_safetensors":False})

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

* **output_dir:** Specifies the path of the folder where the training results of the model (model weights, tokenizer, config files, etc.) will be saved. For example, "output_dir": "bert_model". This folder will be updated as the model trains.

* **overwrite_output_dir:** If set to True, the specified output_dir folder will be overwritten if it already exists. For example, "overwrite_output_dir": True. If set to False, the old model files will be preserved and not overwritten.

* **reprocess_input_data:** If set to True, the data will be reprocessed and fed into the model again. If set to False, the preprocessed data will be used if available. For example, "reprocess_input_data": True.

* **use_cuda:** If set to True, it speeds up the training by using the GPU. If you have a CUDA-supported GPU, keeping this setting enabled will speed up model training. For example, "use_cuda": True.

* **num_train_epochs:** Specifies how many epochs the model will train for. An epoch is a complete cycle of the model going through the dataset. For example, "num_train_epochs": 5 indicates that the model will make 5 complete passes over the dataset.

* **train_batch_size:** Specifies the number of examples to process in each training step. Higher values use more GPU memory but can speed up the training process. For example, "train_batch_size": 64.

* **learning_rate:** Specifies the learning rate used by the optimization algorithm during training. This parameter controls how quickly or slowly the model learns. For example, "learning_rate": 4e-5.

* **weight_decay:** Specifies the amount of penalization applied to the weights to prevent overfitting. Lower values are generally suitable. For example, "weight_decay": 0.01.

* **fp16:** If set to True, the training will be done with float16 (half precision). This can speed up training and reduce memory usage, especially when working with GPUs. However, this setting may not be supported for some models. For example, "fp16": False will train with 32-bit (float32) precision.

* **save_safetensors:** If set to True, the model weights will be saved in .safetensors format. If set to False, the weights will be saved in .bin format.

* **use_early_stopping:** If set to True, the training will stop early when a certain criterion is met. For example, training can be stopped if the loss does not improve after a certain point.

* **early_stopping_patience:** Specifies the number of evaluation cycles before stopping early if no improvement is observed. For example, "early_stopping_patience": 3 will stop training if no improvement is seen after 3 evaluations.

---

**Simpletransformers** automatically uses the appropriate Tokenizer for the specified BERT model. In this example, we are using the 'dbmdz/bert-base-turkish-uncased' model, and its tokenization process happens in a way that is suitable for the language structure of the model. That is, it performs tokenization and model input preparation in a way that is suitable for the Turkish language.

In [12]:
model.train_model(train)

/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/39 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/39 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/39 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/39 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/39 [00:00<?, ?it/s]

(195, 0.15300444861491902)

In [13]:
result, model_outputs, wrong_predictions = model.eval_model(test)

/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

In [14]:
predictions = model_outputs.argmax(axis=1)

In [15]:
actuals = test.labels.values

In [16]:
predictions[:10]

array([3, 0, 2, 2, 3, 0, 4, 4, 1, 2])

In [17]:
actuals[:10]

array([3, 0, 2, 2, 3, 0, 4, 4, 1, 2])

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.9869494290375204

In [19]:
sample = test.iloc[43]['metin']
print(sample)

Kanserojen maddeler, serum ambalajlarına kadar sızdı. İstanbul’da, iki lise öğrencisi yaptığı araştırmayla bu gerçeği ortaya çıkardı. "Sağlık Endüstrilerinde Yapısal Dönüşüm Programı"yla tıbbi cihaz ve malzeme ihtiyacının yüzde 20'sinin, ilaç ihtiyacının ise yüzde 60'ının yerli üretimle karşılanması planlanıyor.


In [20]:
prediction = model.predict([sample])

if prediction[0] ==0:
    print("BİLİM VE TEKNOLOJİ")  # SCIENCE AND TECHNOLOGY
elif prediction[0]==1:
    print("EKONOMİ")             # ECONOMY
elif prediction[0]==2:
    print("SAĞLIK")              # HEALTH
elif prediction[0]==3:
    print("SİYASET")             # POLITICS
else:
    print("SPOR")                # SPORT

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

SAĞLIK


# <font color='purple'> <b>Prediction with Test Data</b><font color='black'>  

In [21]:
# test data import
test_data =pd.read_excel("/kaggle/input/bert-multiclass/test_data.xlsx", names=["kategori","metin"])

In [22]:
test_data

,kategori,metin
0,Spor,İngiltere Premier Lig'in 33. haftasında Liverp...
1,Siyaset,Çanakkale İl Genel Meclisi Başkanı ve AK Parti...
2,Siyaset,Ana muhalefet partisi CHP’nin 12 yıldır başarı...
3,Siyaset,CHP’nin seçimden seçime seçmenini hatırladığın...
4,Spor,Premier Lig'de 2017-2018 ve 2018-2019 sezonlar...
5,Spor,Süper Lig'de 37. hafta mücadelesinde Konyaspor...
6,Ekonomi,ABD Ticaret Bakanı Gina Raimondo da iklim deği...
7,Ekonomi,BD'nin ev sahipliğinde 40 ülke liderinin katıl...
8,Ekonomi,"Artık birçok işletme, modellerinde gölge karbo..."
9,Teknoloji,Apple dün gerçekleştirdiği etkinlikte M1 işlem...


In [23]:
def make_prediction(model, metin):
    
    # Prediction
    prediction = model.predict([metin])
    
    # Print category according to prediction result
    if prediction[0] == 0:
        print("BİLİM VE TEKNOLOJİ")
    elif prediction[0] == 1:
        print("EKONOMİ")
    elif prediction[0] == 2:
        print("SAĞLIK")
    elif prediction[0] == 3:
        print("SİYASET")
    else:
        print("SPOR")

In [24]:
try_1 = test_data.iloc[3]["metin"]

In [25]:
pred_1 = make_prediction(model, try_1)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

SİYASET


In [26]:
try_2 = test_data.iloc[11]["metin"]

In [27]:
pred_2 = make_prediction(model, try_2)

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

SAĞLIK


<p style="background-color:purple;font-family:newtimeroman;font-size:200%;color:white;text-align:center;border-radius:60px 20px;"><b>THANK YOU!</b></p>